# Chapter 5.3

## Importing Libraries

In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import copy
import tqdm
from hfunc import models
from hfunc import metrics
import time

## Self-created functions

In [3]:
def node_pruning(model, tester_model, x, y, layer_sizes, tol, ignore_cutoff=-1e-2, method='exhaustive', label=None, frac=0.75):

    loss, acc = model.evaluate(x, y, verbose=0, batch_size=256)
    original = model.get_weights()
    weight_len = len(original) - 3
    bas = [acc]
    bls = [loss]
    best_weights = model.get_weights()
    best_acc = 0
    best_loss = 1e20
    ol = loss
    oa = acc
    amounts = []
    places = []
    
    if label:
        y_pred = model.predict(x)
        y_pred_flat = np.argmax(y_pred, axis=1)
        y_pred_class = np.array([1 if y == label else 0 for yp in y_pred_flat])
        y_class = np.array([1 if yp == label else 0 for yp in y])

        oa = (y_pred_class == y_class).mean()

    for layer, size in enumerate(layer_sizes):
        end_not_reached = True
        num_removed = 0
        nodes_removed = []
        if method == 'exhaustive':
            current_pos = 0
            best_change = tol
            best_pos = -1
            improved = False
            while end_not_reached or improved:
                if not(end_not_reached):
                    end_not_reached = True
                    improved = False
                    current_pos = 0
                    size -= 1
                    nodes_removed += [best_pos]
                    best_weights[weight_len - (2*layer+1)][...,best_pos] = 0
                    best_weights[weight_len - 2*layer][best_pos] = 0
                    best_pos = -1
                    ol = best_loss
                    oa = best_acc
                    bas += [best_acc]
                    bls += [best_loss]
                    best_change = tol
                    num_removed += 1
                if current_pos in nodes_removed:
                    current_pos += 1
                    if current_pos - num_removed >= size:
                        end_not_reached = False
                    continue
                w = copy.deepcopy(best_weights)
                w[weight_len - (2*layer+1)][...,current_pos] = 0
                w[weight_len - 2*layer][current_pos] = 0
                tester_model.set_weights(w)
                nl, na = tester_model.evaluate(x, y, verbose=0, batch_size=256)
                if label:
                    y_pred = tester_model2.predict(x)
                    y_pred_flat = np.argmax(y_pred, axis=1)
                    y_pred_class = np.array([1 if yp == label else 0 for yp in y_pred_flat])
                    na = (y_pred_class == y_class).mean()
                    change = frac*(na - oa) + (1-frac)*(ol - nl) 
                else:
                    change = ol - nl
                if change >= best_change:
                    best_change = change
                    best_pos = current_pos
                    improved = True
                    best_acc = na
                    best_loss = nl
                current_pos += 1
                if current_pos - num_removed >= size:
                    end_not_reached = False
        elif method == 'greedy':
            nodes_to_estimate = list(np.arange(size))
            current_pos = nodes_to_estimate[0]
            idx = 0
            while end_not_reached:
                w = copy.deepcopy(best_weights)
                w[weight_len - (2*layer+1)][...,current_pos] = 0
                w[weight_len - 2*layer][current_pos] = 0
                tester_model.set_weights(w)
                nl, na = tester_model.evaluate(x, y, verbose=0, batch_size=256)
                
                if label:
                    y_pred = tester_model2.predict(x)
                    y_pred_flat = np.argmax(y_pred, axis=1)
                    y_pred_class = np.array([1 if yp == label else 0 for yp in y_pred_flat])
                    na = (y_pred_class == y_class).mean()
                    change = frac*(na - oa) + (1-frac)*(ol - nl) 
                else:
                    change = ol - nl
                    
                if change >= tol:
                    oa = na
                    ol = nl
                    size -= 1
                    nodes_removed += [current_pos]
                    nodes_to_estimate.remove(current_pos)
                    best_weights[weight_len - (2*layer+1)][..., current_pos] = 0
                    best_weights[weight_len - 2*layer][current_pos] = 0
                    bas += [oa]
                    bls += [ol]
                    num_removed += 1
                    idx = 0
                elif ol - nl <= ignore_cutoff:
                    size -= 1
                    nodes_to_estimate.remove(current_pos)
                else:
                    idx += 1
                if idx >= size:
                    end_not_reached = False
                else:
                    current_pos = nodes_to_estimate[idx]
        amounts.append(num_removed)
        places.append(nodes_removed)

    return best_weights, bas, bls, amounts, places

## Convolutional Neural Network

### CIFAR 10

In [79]:
np.random.seed(2020)

In [80]:
cifar = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [81]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(x_train, y_train, epochs=5)
or_loss, or_acc = model.evaluate(x_test, y_test)

tol = -1e-5
ig_cut = -1e-3
layer_sizes = [64, 356, 128, 64, 32]
K = 10

best_weigths, _, _, am_loss_prune, pl_loss_prune = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy')
tester_model.set_weights(best_weights)

loss_pruned_loss, loss_pruned_acc = tester_model.evaluate(x_test)

y_pred = tester_model.predict(x_test)
loss_pruned_class_acc = []
yp = np.argmax(y_pred, axis=1)
for i in range(K):
    a = np.mean((yp[y_test == i] == y_test[y_test == i]))
    loss_pruned_class_acc.append(a)

    
class_pruned_accs = np.zeros(K)
class_pruned_losses = np.zeros(K)
class_pruned_class_acc = np.zeros((K,K))
class_pruned_place = []
class_pruned_amounts = []

for k in range(K):
    best_weigths, _, _, tmp_am, tmp_pl = node_pruning(model, tester_model, x_val, y_val, layer_sizes, tol, ig_cut, method='greedy', layer=k, frac=1)
    
    class_pruned_amounts += [tmp_am]
    class_pruned_places += [tmp_pl]
    
    tester_model.set_weights(best_weights)

    class_pruned_losses[k], class_pruned_accs[k] = tester_model.evaluate(x_test)

    y_pred = tester_model.predict(x_test)
    yp = np.argmax(y_pred, axis=1)
    for i in range(K):
        a = np.mean((yp[y_test == i] == y_test[y_test == i]))
        class_pruned_class_acc[k,i] = a